## ✓ 튜닝해볼 수 있는 모델 클래스 인자
- lightgbm
  - max_depth : 의사 결정 나무의 깊이, 정수 사용
  - learning_rate : 한 스텝에 이동하는 양을 결정하는 파라미터, 보통 0.0001~0.1 사이의 실수 사용
  - n_estimators : 사용하는 개별 모델의 개수, 보통 50~100 이상의 정수 사용
  - num_leaves : 하나의 LightGBM 트리가 가질 수 있는 최대 잎의 수
  - boosting_type : 부스팅 방식, gbdt, rf 등의 문자열 입력

## ✓ 시도해볼 수 있는 방법
- 기존에 있는 데이터의 피처를 모델을 보다 잘 표현할 수 있는 형태로 처리하기 (피처 엔지니어링)
- LGBMRegressor, XGBRegressor, RandomForestRegressor 세 가지 이상의 다양한 모델에 대해 하이퍼 파라미터 튜닝하기
- 다양한 하이퍼 파라미터에 대해 그리드 탐색을 시도해서 최적의 조합을 찾아보기
- Baseline 커널에서 활용했던 블렌딩 방법 활용하기

## 프로젝트 루브릭

- 데이터 전처리, 모델학습, 예측의 전체 과정을 거쳐 캐글 submission까지 전과정이 성공적으로 진행되었는가?
  1. 캐글 데이터분석 전과정이 성공적으로 진행되었는가?
     - 작성한 노트북을 캐글에 제출했다.
- 제출된 노트북이 캐글 커널로 사용될 수 있을 만큼 전처리, 학습, 최적화 진행 과정이 체계적으로 기술되었는가?
  2. 전처리, 학습과정 및 결과에 대한 설명이 시각화를 포함하여 체계적으로 진행되었는가?
     - 제출한 주피터노트북 파일이 캐글 커널 환경에서도 에러 없이 동작하며, 전처리, 학습, 최적화 진행 과정이 체계적으로 기술되었다.
- 다양한 피처 엔지니어링과 하이퍼 파라미터 튜닝 등의 최적화 기법을 통해 캐글 리더보드의 Private score 기준 110000 이하의 점수를 얻었는가?
  3. 회귀모델 예측정확도가 기준 이상 높게 나왔는가?
     - 다양한 피처 엔지니어링과 하이퍼 파라미터 튜닝 등의 최적화 기법을 통해 캐글 리더보드의 Private score 기준 110000 이하의 점수를 얻었다.train_data_path


In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Aiffel environment
# data_dir = '~/aiffel/kaggle_kakr_housing/data'
# Kaggle environment
# data_dir = '/kaggle/input/2019-2nd-ml-month-with-kakr' # 해당 카글 데이터는 test가 6555개임..
data_dir = '/kaggle/input/myfirstplayground'
train_data_path = join(data_dir, 'train.csv')
sub_data_path = join(data_dir, 'test.csv')

data = pd.read_csv(train_data_path)
sub = pd.read_csv(sub_data_path)

print(f'train data dim : {data.shape}')
print(data.columns)

print(f'sub data dim : {sub.shape}')
print(sub.columns)

train data dim : (15035, 21)
Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')
sub data dim : (6468, 20)
Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


In [3]:
# price는 target 값으로 쓰이니깐 해당 price column을 y에 대입하고 삭제해준다.
y = data['price']
del data['price']
print(y)
print(data.columns)

0         221900.0
1         180000.0
2         510000.0
3         257500.0
4         291850.0
           ...    
15030     610685.0
15031    1007500.0
15032     360000.0
15033     400000.0
15034     325000.0
Name: price, Length: 15035, dtype: float64
Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


In [4]:
# 주어진 train.csv와 test.csv를 합하여 사용한다.
train_len = len(data)
data = pd.concat((data, sub), axis=0)
data.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


In [5]:
# id 값도 필요없으니 빼준다. 하지만 예측 결과를 제출할 때를 대비하여 sub_id 변수에 id 칼럼을 저장한다.
sub_id = data['id'][train_len:]
del data['id']
data.columns

Index(['date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [6]:
# date column에서 year, month만 추출하여 integer로 바꿔서 사용한다.
# data['date'] = data['date'].apply(lambda x: int(x[:6]))
data['year'] = data['date'].apply(lambda x: int(x[:4]))
data['month'] = data['date'].apply(lambda x: int(x[4:6]))
del data['date']
data.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'year', 'month'],
      dtype='object')

In [7]:
# 한쪽으로 치우친 그래프 로그 변환
skew_columns = ['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_lot15', 'sqft_living15']
for c in skew_columns:
    data[c] = np.log1p(data[c].values)

print('done...')

done...


In [8]:
# train_len을 인덱스로 활용해서 :train_len까지는 학습 데이터, 즉 x에 저장
# train_len: 부터는 실제로 추론을 해야 하는 테스트 데이터, 즉 sub 변수에 저장
# sub = data.iloc[:6468, :]
# x = data.iloc[6468:, :]
sub = data.iloc[train_len:, :]
x = data.iloc[:train_len, :]
sub.shape, x.shape

((6468, 20), (15035, 20))

In [9]:
def get_cv_score(models):
    kfold = KFold(n_splits=5).get_n_splits(x.values)
    for m in models:
        CV_score = np.mean(cross_val_score(m['model'], X=x.values, y=y, cv=kfold))
        print(f"Model: {m['name']}, CV score:{CV_score:.4f}")

print('define get_cv_score func...')

define get_cv_score func...


In [10]:
def AveragingBlending(models, x, y, sub_x):
    for m in models : 
        m['model'].fit(x.values, y)
    
    predictions = np.column_stack([
        m['model'].predict(sub_x.values) for m in models
    ])
    return np.mean(predictions, axis=1)

print('define AveragingBlending func...')

define AveragingBlending func...


In [11]:
def my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=5):
    grid_model = GridSearchCV(model, param_grid=param_grid, \
                        scoring='neg_mean_squared_error', \
                        cv=5, verbose=verbose, n_jobs=n_jobs)

    grid_model.fit(train, y)
    
    # params = grid_model.cv_results_['params']
    # score = grid_model.cv_results_['mean_test_score']

    # results = pd.DataFrame(params)
    # results['score'] = score
    # results['RMSLE'] = np.sqrt(-1 * results['score'])
    # results = results.sort_values('RMSLE')

    # return results
    return grid_model.best_estimator_

print('define my_GridSearch func...')

define my_GridSearch func...


In [12]:
def save_submission(model, train, y, test, model_name, rmsle):
    model.fit(train, y)
    prediction = model.predict(test)
    prediction = np.expm1(prediction)

    data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'
    submission_path = join(data_dir, 'sample_submission.csv')

    submission = pd.read_csv(submission_path)
    submission['price'] = prediction

    submission_csv_path = '{}/submission_{}_RMSLE_{}.csv'.format(data_dir, model_name, rmsle)
    submission.to_csv(submission_csv_path, index=False)
    print('{} saved!'.format(submission_csv_path))

print('define save_submission func...')

define save_submission func...


In [13]:
# define models

gboost = GradientBoostingRegressor(random_state=2019)
xgboost = xgb.XGBRegressor(random_state=2019)
lightgbm = lgb.LGBMRegressor(random_state=2019)
rf = RandomForestRegressor(random_state=2019)

models = [{'model': gboost, 'name': 'GradientBoosting'}, 
          {'model': xgboost, 'name': 'XGBoost'},
          {'model': lightgbm, 'name': 'LightGBM'},
          {'model': rf, 'name': 'RandomForest'}]

models
# get_cv_score(models)

[{'model': GradientBoostingRegressor(random_state=2019),
  'name': 'GradientBoosting'},
 {'model': XGBRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, random_state=2019, ...),
  'name': 'XGBoost'},
 {'model': LGBMRegressor(random_state=2019), 'name': 'LightGBM'},
 {'model': RandomForestRegressor(random_state

In [ ]:
param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [10, 20, 30], 'min_samples_split': [2, 5, 10]}
param_grid_xgb = {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7], 'learning_rate': [0.01, 0.05, 0.1]}
param_grid_lgb = {'n_estimators': [100, 200, 300], 'max_depth': [1, 10, 20], 'num_leaves': [31, 50, 70], 'learning_rate': [0.01, 0.05, 0.1]}

grid_rf = my_GridSearch(rf, x, y, param_grid_rf)
grid_xgb = my_GridSearch(xgboost, x, y, param_grid_xgb)
grid_lgb = my_GridSearch(lightgbm, x, y, param_grid_lgb)

print(grid_rf)
print(grid_xgb)
print(grid_lgb)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 81 candidates, totalling 405 fits


In [ ]:
y_pred = AveragingBlending(models, x, y, sub)

In [ ]:
sub = pd.DataFrame(data={'id': sub_id, 'price': y_pred})
sub.to_csv('/kaggle/working/submission.csv', index=False)